In [ ]:
#|default_exp metamorfoze

# A dive into deltaE CIE2000

> What you always wanted to know about CIE 2000 color difference computations. 

The most recent Metamorfoze standard prescribes that for assessing color differences one should use the newest CIE2000 deltaE formula, **with the addition that SL=1**. So far I managed to avoid the complexities of CIE2000, but it seems that now we can longer ignore its inner workings. Here is the formula for the CIE 2000 color difference. 

$$ \Delta E^*_{00} = \sqrt{\left(\frac{\Delta L'}{k_L \color{red}{S_L} }\right)^2 + \left(\frac{\Delta C'}{k_C S_C}\right)^2 + \left(\frac{\Delta H'}{k_H S_H }\right)^2  + \left(R_T \left(\frac{\Delta C'}{K_C S_C}\right) \left(\frac{\Delta H'}{k_H S_H}\right)\right)} $$

The important thing to note here is that we find indeed a coefficient *SL* in the denominator of the first term. Note that this coefficient is not constant but a function that depends on the CIELAB inputs! Further specification of the variables in this formula in terms of CIELAB input values would require 27 more lines of math. If you are curious, see the **Math > DeltaE (CIE 2000)** section of [www.brucelindbloom.com](http://www.brucelindbloom.com/). 

To study the effect of setting SL=1, I created the `delta_E_CIE2000_adapted()` function. Let's see if we can now replicate the BasICColor Metamorfoze (SL=1) report values... 

In [ ]:
from colorchecker2cielab import delta_E_CIE2000_adapted, read_basiccolor_data 
import pandas as pd

In [ ]:
# read BasICColor report data 
bcol_data = '/home/frank/Work/DATA/colorchecker2cielab-data/BasiCColor_sk-c-1833.xlsx'
bcol_df = read_basiccolor_data(bcol_data)

In [ ]:
# select CIELAB values needed to compute deltaE 
ref_LABs = bcol_df.loc[:, ['ref_L*', 'ref_a*', 'ref_b*']].values
bcol_LABs = bcol_df.loc[:, ['bcol_L*', 'bcol_a*', 'bcol_b*']].values

bcol_deltaE_SL1 = bcol_df.loc[:, ['bcol_ΔE*00_(SL=1)']]

In [ ]:
computed_deltaE_SL1 = delta_E_CIE2000_adapted(ref_LABs, bcol_LABs, metamorfoze_SL_is_one=True)
compare_df = bcol_deltaE_SL1
compare_df['computed'] = computed_deltaE_SL1
pd.options.display.float_format = '{:,.2f}'.format 
print(compare_df.to_string());

     bcol_ΔE*00_(SL=1)  computed
A1                0.77      0.76
B1                1.05      1.05
C1                1.97      1.97
D1                0.68      0.68
E1                1.00      1.00
F1                2.24      2.24
G1                0.67      0.67
H1                0.98      0.98
I1                2.05      2.05
J1                0.64      0.63
K1                1.01      1.02
L1                2.70      2.70
M1                0.94      0.93
N1                0.69      0.69
A2                2.28      2.28
B2                1.77      1.77
C2                1.19      1.19
D2                1.03      1.03
E2                1.04      1.05
F2                1.55      1.55
G2                0.82      0.82
H2                0.49      0.49
I2                1.11      1.11
J2                1.28      1.28
K2                0.74      0.75
L2                1.73      1.73
M2                1.48      1.49
N2                0.94      0.94
A3                1.09      1.09
B3        

Conclusion is that we can indeed replicate the CIE2000(SL=1) deltaE values. 

**However, I can't see yet why we should follow the Metamorfoze Standard recommendation...** 

In [ ]:
#|export 

#from __future__ import annotations
from colour.utilities import as_float, to_domain_100, tsplit 
import numpy as np

#from colour.algebra import euclidean_distance
#from colour.hints import ArrayLike, NDArrayFloat


In [ ]:
#|export 

def delta_E_CIE2000_adapted(Lab_1, Lab_2, textiles=False, metamorfoze_SL_is_one=False):
    """ 
    Return the difference :math:`\\Delta E_{00}` between two given
    *CIE L\\*a\\*b\\** colourspace arrays using *CIE 2000* recommendation. 

    ADAPTED BY FRANK LIGTERINK TO INCLUDE THE CURIOUS OPTION: `metamorfoze_SL_is_one` 

    Parameters
    ----------
    Lab_1
        *CIE L\\*a\\*b\\** colourspace array 1.
    Lab_2
        *CIE L\\*a\\*b\\** colourspace array 2.
    textiles
        Textiles application specific parametric factors.
        :math:`k_L=2,\\ k_C=k_H=1` weights are used instead of
        :math:`k_L=k_C=k_H=1`.

    Returns
    -------
    :class:`numpy.ndarray`
        Colour difference :math:`\\Delta E_{00}`.

    Notes
    -----
    +------------+-----------------------+-------------------+
    | **Domain** | **Scale - Reference** | **Scale - 1**     |
    +============+=======================+===================+
    | ``Lab_1``  | ``L_1`` : [0, 100]    | ``L_1`` : [0, 1]  |
    |            |                       |                   |
    |            | ``a_1`` : [-100, 100] | ``a_1`` : [-1, 1] |
    |            |                       |                   |
    |            | ``b_1`` : [-100, 100] | ``b_1`` : [-1, 1] |
    +------------+-----------------------+-------------------+
    | ``Lab_2``  | ``L_2`` : [0, 100]    | ``L_2`` : [0, 1]  |
    |            |                       |                   |
    |            | ``a_2`` : [-100, 100] | ``a_2`` : [-1, 1] |
    |            |                       |                   |
    |            | ``b_2`` : [-100, 100] | ``b_2`` : [-1, 1] |
    +------------+-----------------------+-------------------+

    -   Parametric factors :math:`k_L=k_C=k_H=1` weights under
        *reference conditions*:

        -   Illumination: D65 source
        -   Illuminance: 1000 lx
        -   Observer: Normal colour vision
        -   Background field: Uniform, neutral gray with :math:`L^*=50`
        -   Viewing mode: Object
        -   Sample size: Greater than 4 degrees
        -   Sample separation: Direct edge contact
        -   Sample colour-difference magnitude: Lower than 5.0
            :math:`\\Delta E_{00}`
        -   Sample structure: Homogeneous (without texture)

    References
    ----------
    :cite:`Melgosa2013b`, :cite:`Sharma2005b`

    Examples
    --------
    >>> Lab_1 = np.array([100.00000000, 21.57210357, 272.22819350])
    >>> Lab_2 = np.array([100.00000000, 426.67945353, 72.39590835])
    >>> delta_E_CIE2000(Lab_1, Lab_2)  # doctest: +ELLIPSIS
    94.0356490...
    >>> Lab_2 = np.array([50.00000000, 426.67945353, 72.39590835])
    >>> delta_E_CIE2000(Lab_1, Lab_2)  # doctest: +ELLIPSIS
    100.8779470...
    >>> delta_E_CIE2000(Lab_1, Lab_2, textiles=True)  # doctest: +ELLIPSIS
    95.7920535...
    """

    L_1, a_1, b_1 = tsplit(to_domain_100(Lab_1))
    L_2, a_2, b_2 = tsplit(to_domain_100(Lab_2))

    k_L = 2 if textiles else 1
    k_C = 1
    k_H = 1

    C_1_ab = np.hypot(a_1, b_1)
    C_2_ab = np.hypot(a_2, b_2)

    C_bar_ab = (C_1_ab + C_2_ab) / 2
    C_bar_ab_7 = C_bar_ab**7

    G = 0.5 * (1 - np.sqrt(C_bar_ab_7 / (C_bar_ab_7 + 25**7)))

    a_p_1 = (1 + G) * a_1
    a_p_2 = (1 + G) * a_2

    C_p_1 = np.hypot(a_p_1, b_1)
    C_p_2 = np.hypot(a_p_2, b_2)

    h_p_1 = np.where(
        np.logical_and(b_1 == 0, a_p_1 == 0),
        0,
        np.degrees(np.arctan2(b_1, a_p_1)) % 360,
    )
    h_p_2 = np.where(
        np.logical_and(b_2 == 0, a_p_2 == 0),
        0,
        np.degrees(np.arctan2(b_2, a_p_2)) % 360,
    )

    delta_L_p = L_2 - L_1

    delta_C_p = C_p_2 - C_p_1

    h_p_2_s_1 = h_p_2 - h_p_1
    C_p_1_m_2 = C_p_1 * C_p_2
    delta_h_p = np.select(
        [
            C_p_1_m_2 == 0,
            np.fabs(h_p_2_s_1) <= 180,
            h_p_2_s_1 > 180,
            h_p_2_s_1 < -180,
        ],
        [
            0,
            h_p_2_s_1,
            h_p_2_s_1 - 360,
            h_p_2_s_1 + 360,
        ],
    )

    delta_H_p = 2 * np.sqrt(C_p_1_m_2) * np.sin(np.deg2rad(delta_h_p / 2))

    L_bar_p = (L_1 + L_2) / 2

    C_bar_p = (C_p_1 + C_p_2) / 2

    a_h_p_1_s_2 = np.fabs(h_p_1 - h_p_2)
    h_p_1_a_2 = h_p_1 + h_p_2
    h_bar_p = np.select(
        [
            C_p_1_m_2 == 0,
            a_h_p_1_s_2 <= 180,
            np.logical_and(a_h_p_1_s_2 > 180, h_p_1_a_2 < 360),
            np.logical_and(a_h_p_1_s_2 > 180, h_p_1_a_2 >= 360),
        ],
        [
            h_p_1_a_2,
            h_p_1_a_2 / 2,
            (h_p_1_a_2 + 360) / 2,
            (h_p_1_a_2 - 360) / 2,
        ],
    )

    T = (
        1
        - 0.17 * np.cos(np.deg2rad(h_bar_p - 30))
        + 0.24 * np.cos(np.deg2rad(2 * h_bar_p))
        + 0.32 * np.cos(np.deg2rad(3 * h_bar_p + 6))
        - 0.20 * np.cos(np.deg2rad(4 * h_bar_p - 63))
    )

    delta_theta = 30 * np.exp(-(((h_bar_p - 275) / 25) ** 2))

    C_bar_p_7 = C_bar_p**7
    R_C = 2 * np.sqrt(C_bar_p_7 / (C_bar_p_7 + 25**7))

    L_bar_p_2 = (L_bar_p - 50) ** 2 

    S_L = 1 + ((0.015 * L_bar_p_2) / np.sqrt(20 + L_bar_p_2))

    # ADDED BY FL 
    if metamorfoze_SL_is_one: 
        S_L = 1 
    
    S_C = 1 + 0.045 * C_bar_p

    S_H = 1 + 0.015 * C_bar_p * T

    R_T = -np.sin(np.deg2rad(2 * delta_theta)) * R_C

    d_E = np.sqrt(
        (delta_L_p / (k_L * S_L)) ** 2
        + (delta_C_p / (k_C * S_C)) ** 2
        + (delta_H_p / (k_H * S_H)) ** 2
        + R_T * (delta_C_p / (k_C * S_C)) * (delta_H_p / (k_H * S_H))
    )

    return as_float(d_E)
